## LBC group management


In [189]:
import collections
from dataclasses import dataclass
import pandas as pd
import numpy as np
from collections import namedtuple
group_chars = [c for c in 'ABCDEFGHIJKLMNOP']
requirement = namedtuple('requirement', 'target min max')
sc_group_size = requirement(6,5,6)
sc_singles_count = requirement(2,0,4)
sc_couple_count = requirement(3,3,4)
lb_group_size = requirement(6,5,7)
lb_singles_count = requirement(2,0,3)
lb_couple_count = requirement(2,2,3)


### import historical groupings
- columns are groupings, of either type 'lb' or 'sc'
- rows are members
- character denotes membership in a group. same letter same group. null for no involvement

In [190]:
column_names = ['index', 'size'] + list(f'{"sc" if i%2 else "lb"}{i//2}'
                             for i in range(29)
                             ) + ['elb_block', 'llb_block', 'sc_block']
group_history = pd.read_csv("LBC_group_history.csv", names=column_names, header=None)

display(group_history.head(5))

,index,size,lb0,sc0,lb1,sc1,lb2,sc2,lb3,sc3,...,lb11,sc11,lb12,sc12,lb13,sc13,lb14,elb_block,llb_block,sc_block
0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X,X
1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,A,NaN,X,X,X
2,2,1,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X,X
3,3,2,D,NaN,F,NaN,G,NaN,NaN,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,X,X
4,4,1,A,E,F,C,I,B,F,C,...,NaN,NaN,NaN,C,NaN,D,NaN,X,NaN,X


### measure group sizes
- get total group sizes using weights from size column


In [191]:
weights = group_history.iloc[:, 1].values

group_sizes = pd.DataFrame([
    [
        np.sum([weights[i] if value == group_char else 0 for i, value in enumerate(group_history[column])])
        for group_char in group_chars
    ] for column in group_history.columns[2:-3]
],
columns=group_chars,
index=[column for column in group_history.columns[2:-3]],
).replace(0, np.nan)

display(group_sizes.tail(5))


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
lb12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sc12,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lb13,4.0,6.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sc13,10.0,8.0,13.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lb14,9.0,6.0,4.0,6.0,6.0,6.0,6.0,6.0,8.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN


### create met_before lookup dict
- key: group history index
- value: list of group history indexes


In [192]:
all_groups = list()
for col_name, grouping in group_history.iteritems():
    for group_char in set(grouping):
        if group_char in group_chars:
            all_groups.append(group_history.index[group_history[col_name] == group_char].tolist())

met_before = {i: set([i,]) for i, groups in group_history.iterrows()}
for group in all_groups:
    for i in group:
        for j in group:
            met_before[i].add(j)


In [200]:
met_before_df = pd.DataFrame.from_dict(met_before, orient='index')
display(met_before_df)

# Export Met Before data to csv
met_before_df.to_csv("met_before.csv", sep='\t')


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0,66.0,70.0,8.0,76.0,44.0,49.0,20.0,116.0,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,66.0,99.0,36.0,130.0,113.0,49.0,115.0,20.0,116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,129,2.0,3.0,66.0,76.0,109.0,87.0,91.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,3.0,66.0,4.0,37.0,103.0,27.0,6.0,74.0,76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,128,130.0,3.0,4.0,5.0,6.0,11.0,19.0,20.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,66,75.0,12.0,15.0,17.0,52.0,118.0,26.0,91.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,128,66.0,130.0,4.0,103.0,8.0,74.0,43.0,45.0,113.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,64,129.0,2.0,6.0,71.0,106.0,76.0,80.0,18.0,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,128,1.0,130.0,4.0,5.0,9.0,11.0,18.0,20.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### data sanity check
- expect 10-25 unique groups in each round
- groups around 4-8 in size
- max possible group memberships is 29
- 20-50% of members never active before


In [194]:
# filter row data
# [list(filter(lambda x: x>0, grouping)) for i, grouping in group_sizes.iterrows()]

group_counts = [np.sum(len(list(filter(lambda x: x>0, grouping)))) for i, grouping in group_sizes.iterrows()]

features = pd.DataFrame(
    [
        [
            # number of groups
            np.mean(group_counts),
            np.min(group_counts),
            np.max(group_counts),
        ],
        [
            # group size
            np.mean([np.nanmean(group_sizes)]),
            np.min([np.nanmin(group_sizes)]),
            np.max([np.nanmax(group_sizes)]),
        ],
        [
            # socializing
            np.mean([len(met_set) for i, met_set in met_before.items()]),
            np.min([len(met_set) for i, met_set in met_before.items()]),
            np.max([len(met_set) for i, met_set in met_before.items()]),
        ],
    ],
    columns=['mean', 'min', 'max'],
    index=['number of groups', 'group size head count', 'met counts'],
)

display(features)

,mean,min,max
number of groups,4.448276,0.0,11.0
group size head count,6.100775,1.0,13.0
met counts,11.287879,1.0,48.0


# Create next SC and LB groupings
- confirm group requirements


In [195]:
display(pd.DataFrame([
    sc_group_size, sc_singles_count, sc_couple_count,
    lb_group_size, lb_singles_count, lb_couple_count,
],
    columns=['target', 'min', 'max'],
    index=['sc group size', 'sc singles count', 'sc couple count',
           'lb group size', 'lb singles count', 'lb couple count',],
))

,target,min,max
sc group size,6,5,6
sc singles count,2,0,4
sc couple count,3,3,4
lb group size,6,5,7
lb singles count,2,0,3
lb couple count,2,2,3


- create groups
- disperse indexes by order of met counts
- check member requirements
- check total size limits

In [196]:
# sort indexes by weight (couples first), then met_before count (most first_
ordered_indexes = collections.OrderedDict(sorted(met_before.items(), key=lambda met_set: (-weights[met_set[0]], -len(met_set[1]))))
# for i, met_set in ordered_indexes.items():
#     print(f"{i} has met {met_set}")

In [197]:
indexes_list = list(ordered_indexes.keys())

min_groups_count = {
    "sc": 5,  # np.sum(weights) / sc_group_size.target
    "elb": 3,  # np.sum(weights) / lb_group_size.max
    "llb": 5,  # np.sum(weights) / lb_group_size.max
}

@dataclass
class NewGroup:
    members: list
    singles: int
    couples: int
    size: int
    kind: str

def i_can_join(asking_index: int, new_group: NewGroup, please=False):
    if asking_index in new_group.members:
        raise Exception("already a member of this group!")

    if new_group.size >= (lb_group_size.max if new_group.kind.endswith("lb") else sc_group_size.max):
        return False

    if weights[asking_index] == 1 and \
            new_group.singles >= (lb_singles_count.max if new_group.kind.endswith("lb") else sc_singles_count.max):
        return False

    if weights[asking_index] == 2 and \
            new_group.couples >= (lb_couple_count.max if new_group.kind.endswith("lb") else sc_couple_count.max):
        return False

    if please != "pretty please":
        if len(set(new_group.members).intersection(met_before[asking_index])) > 2:
            return False

    if not please:
        if len(set(new_group.members).intersection(met_before[asking_index])) > 0:
            return False

        if new_group.size >= (lb_group_size.target if new_group.kind.endswith("lb") else sc_group_size.target):
            return False
        if weights[asking_index] == 1 and new_group.singles >= (lb_singles_count.target if new_group.kind.endswith("lb") else sc_singles_count.target):
            return False
        if weights[asking_index] >= 2 and new_group.couples >= (lb_couple_count.target if new_group.kind.endswith("lb") else sc_couple_count.target):
            return False

    if len(set(new_group.members).intersection(met_before[asking_index])) > 0:
        those_meeting_the_met = set(new_group.members).intersection(met_before[asking_index])
        if i is not 119:
            print(f'index {i} has met with {those_meeting_the_met} and will see them again')

    return True


def populate_grouping(grouping, group_kind):
    placed_indexes = set()
    displaced_indexes = set()
    for index in indexes_list:
        this_index_placed = False

        if "X" == group_history[f"{group_kind}_block"].values[index]:
            # print(f'index {index} found {group_history[f"{group_kind}_block"].values[index]}')
            continue

        for group_i, group in enumerate(grouping):

            #  fill an empty group
            if len(group.members) == 0:
                assert(group.size==0)
                grouping[group_i].members.append(index)
                this_index_placed = True
                break

            # join group if not full and unmet members
            if i_can_join(index, group):
                grouping[group_i].members.append(index)
                this_index_placed = True
                break

            # create a new group
            if len(grouping) < min_groups_count[group_kind]:
                grouping.append(NewGroup([index,], 0, 0, 0, group_kind))
                this_index_placed = True
                group_i = len(grouping)-1
                break

        # uncomment to allow going beyond targets
        if not this_index_placed:
            for group_i, group in enumerate(grouping):
                if i_can_join(index, group, please=True):  # can set please="pretty please"
                    grouping[group_i].members.append(index)
                    this_index_placed = True
                    break

        if this_index_placed:
            grouping[group_i].size += weights[index]
            if weights[index] == 1:
                grouping[group_i].singles += 1
            else:
                grouping[group_i].couples += 1
            placed_indexes.add(index)
        else:
            displaced_indexes.add(index)

    return grouping, displaced_indexes


<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/88/nw1f1bfs2fd2ck22dqgjy4540000gn/T/ipykernel_83655/213526220.py:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if i is not 119:


# Results

In [198]:
group_keys, groupings, unfit_lists = ['elb', 'llb', 'sc',], dict(), dict()

for round_group_key in group_keys:

    (groupings[round_group_key], unfit_lists[round_group_key]) = populate_grouping([NewGroup([],0,0,0,round_group_key),], round_group_key)
    print(f"{round_group_key} needed at least {int(min_groups_count[round_group_key])} groups. and ended up creating {len(groupings[round_group_key])} groups")

    display(groupings[round_group_key])

    if unfit_lists[round_group_key]:
        print(f"{len(unfit_lists[round_group_key])} leftover could not be placed")
    else:
        print("Everyone was placed!")

    for group in groupings[round_group_key]:
        for member_index in group.members:
            for member in group.members:
                met_before[member_index].add(member)


elb needed at least 3 groups. and ended up creating 3 groups


[NewGroup(members=[47, 95, 63, 105], singles=1, couples=3, size=7, kind='elb'),
 NewGroup(members=[72, 97, 68, 25], singles=1, couples=3, size=7, kind='elb'),
 NewGroup(members=[84, 7, 91, 42, 54], singles=3, couples=2, size=7, kind='elb')]

1 leftover could not be placed
llb needed at least 5 groups. and ended up creating 5 groups


[NewGroup(members=[57, 117, 34], singles=0, couples=3, size=7, kind='llb'),
 NewGroup(members=[113, 111, 98, 74], singles=1, couples=3, size=7, kind='llb'),
 NewGroup(members=[65, 39, 123, 112], singles=1, couples=3, size=7, kind='llb'),
 NewGroup(members=[6, 62, 75, 93], singles=2, couples=2, size=6, kind='llb'),
 NewGroup(members=[119, 101, 4, 70], singles=2, couples=2, size=6, kind='llb')]

Everyone was placed!
sc needed at least 5 groups. and ended up creating 5 groups


[NewGroup(members=[113, 5, 90], singles=0, couples=3, size=6, kind='sc'),
 NewGroup(members=[36, 46, 55], singles=0, couples=3, size=6, kind='sc'),
 NewGroup(members=[122, 120, 40], singles=0, couples=3, size=6, kind='sc'),
 NewGroup(members=[99, 117, 105, 75], singles=2, couples=2, size=6, kind='sc'),
 NewGroup(members=[116, 20, 130, 9, 54], singles=4, couples=1, size=6, kind='sc')]

2 leftover could not be placed


In [199]:
# Jean

jean = 4
print("all the people Jean met before")
print(sorted(list(met_before[jean])))


all the people Jean met before
[3, 4, 5, 6, 11, 19, 20, 25, 27, 36, 38, 42, 46, 47, 50, 52, 61, 66, 67, 70, 75, 76, 82, 91, 92, 93, 94, 95, 96, 97, 101, 105, 108, 110, 113, 115, 116, 117, 119, 120, 128, 130]
